# Streaming with DSPy

This notebook demonstrates how to implement streaming responses with DSPy for real-time AI applications.

## What You'll Learn:
- Setting up streaming responses with DSPy
- Implementing real-time text generation
- Building streaming chat interfaces
- Handling streaming with different LM providers
- Performance optimization for streaming applications

Based on the DSPy tutorial: [Streaming](https://dspy.ai/tutorials/streaming/)

## Setup and Imports

In [ ]:
import os
import sys
sys.path.append('../../')

import dspy
from utils import setup_default_lm, print_step, print_result, print_error
from dotenv import load_dotenv
import asyncio
import time
from typing import Generator, AsyncGenerator, List, Dict, Any
import json
from datetime import datetime
import threading
from queue import Queue
import functools

# Load environment variables
load_dotenv('../../.env')

## Language Model Configuration for Streaming

In [ ]:
print_step("Setting up Language Model", "Configuring DSPy for streaming responses")

try:
    # Configure with streaming-capable model
    lm = setup_default_lm(
        provider="openai", 
        model="gpt-4o-mini", 
        max_tokens=1000,
        stream=True  # Enable streaming
    )
    dspy.configure(lm=lm)
    print_result("Streaming-enabled language model configured successfully!", "Status")
except Exception as e:
    print_error(f"Failed to configure streaming model: {e}")
    # Fallback to non-streaming for demo
    try:
        lm = setup_default_lm(provider="openai", model="gpt-4o-mini", max_tokens=1000)
        dspy.configure(lm=lm)
        print_result("Non-streaming model configured as fallback")
    except Exception as e2:
        print_error(f"Failed to configure fallback model: {e2}")

## Basic Streaming Implementation

In [ ]:
class StreamingTextGenerator:
    """Basic streaming text generator using DSPy."""
    
    def __init__(self):
        self.predictor = dspy.Predict("prompt -> response")
    
    def generate_stream(self, prompt: str) -> Generator[str, None, None]:
        """Generate streaming response for a given prompt."""
        
        try:
            # For demonstration, we'll simulate streaming by chunking a response
            # In practice, you'd use the actual streaming API from your LM provider
            
            # Get full response first (in real streaming, this would be incremental)
            result = self.predictor(prompt=prompt)
            response_text = result.response
            
            # Simulate streaming by yielding chunks
            words = response_text.split()
            current_chunk = ""
            
            for word in words:
                current_chunk += word + " "
                
                # Yield chunks of 3-5 words
                if len(current_chunk.split()) >= 3:
                    yield current_chunk.strip()
                    current_chunk = ""
                    time.sleep(0.1)  # Simulate network delay
            
            # Yield remaining text
            if current_chunk.strip():
                yield current_chunk.strip()
                
        except Exception as e:
            yield f"Error in streaming: {e}"
    
    def generate_complete_stream(self, prompt: str) -> Dict[str, Any]:
        """Generate streaming response with metadata."""
        
        start_time = time.time()
        chunks = list(self.generate_stream(prompt))
        end_time = time.time()
        
        return {
            'chunks': chunks,
            'full_response': ' '.join(chunks),
            'chunk_count': len(chunks),
            'duration': end_time - start_time,
            'timestamp': datetime.now().isoformat()
        }

# Initialize streaming generator
streaming_generator = StreamingTextGenerator()
print_result("Basic streaming text generator initialized")

## Testing Basic Streaming

In [ ]:
print_step("Testing Basic Streaming", "Demonstrating word-by-word text generation")

test_prompt = "Explain the concept of machine learning in simple terms."

print(f"Prompt: {test_prompt}")
print("\nStreaming Response:")
print("-" * 50)

# Demonstrate real-time streaming
full_response = ""
chunk_count = 0

for chunk in streaming_generator.generate_stream(test_prompt):
    print(chunk, end=" ", flush=True)
    full_response += chunk + " "
    chunk_count += 1

print("\n" + "-" * 50)
print_result(f"Streaming completed in {chunk_count} chunks")
print_result(f"Full response length: {len(full_response)} characters")

## Advanced Streaming with DSPy Modules

In [ ]:
class StreamingSignatures:
    """DSPy signatures optimized for streaming."""
    
    class StreamingQA(dspy.Signature):
        """Question answering optimized for streaming response."""
        question = dspy.InputField(desc="User's question")
        answer = dspy.OutputField(desc="Comprehensive answer that can be streamed")
    
    class StreamingStoryGeneration(dspy.Signature):
        """Story generation optimized for streaming."""
        prompt = dspy.InputField(desc="Story prompt or theme")
        story = dspy.OutputField(desc="Engaging story that unfolds progressively")
    
    class StreamingExplanation(dspy.Signature):
        """Technical explanation optimized for streaming."""
        topic = dspy.InputField(desc="Topic to explain")
        audience = dspy.InputField(desc="Target audience level")
        explanation = dspy.OutputField(desc="Clear, progressive explanation")

class AdvancedStreamingModule(dspy.Module):
    """Advanced DSPy module with streaming capabilities."""
    
    def __init__(self):
        super().__init__()
        self.qa_module = dspy.ChainOfThought(StreamingSignatures.StreamingQA)
        self.story_module = dspy.ChainOfThought(StreamingSignatures.StreamingStoryGeneration)
        self.explanation_module = dspy.ChainOfThought(StreamingSignatures.StreamingExplanation)
        
        # Streaming configuration
        self.chunk_size = 5  # words per chunk
        self.delay = 0.08    # seconds between chunks
    
    def stream_qa(self, question: str) -> Generator[Dict[str, Any], None, None]:
        """Stream question-answering response with reasoning."""
        
        # Generate reasoning and answer
        result = self.qa_module(question=question)
        
        # Stream reasoning first
        if hasattr(result, 'reasoning') and result.reasoning:
            yield {
                'type': 'reasoning_start',
                'content': 'Let me think about this...',
                'timestamp': datetime.now().isoformat()
            }
            
            for chunk in self._chunk_text(result.reasoning):
                yield {
                    'type': 'reasoning',
                    'content': chunk,
                    'timestamp': datetime.now().isoformat()
                }
                time.sleep(self.delay)
        
        # Stream answer
        yield {
            'type': 'answer_start',
            'content': 'Here\'s my answer:',
            'timestamp': datetime.now().isoformat()
        }
        
        for chunk in self._chunk_text(result.answer):
            yield {
                'type': 'answer',
                'content': chunk,
                'timestamp': datetime.now().isoformat()
            }
            time.sleep(self.delay)
        
        yield {
            'type': 'complete',
            'content': '',
            'timestamp': datetime.now().isoformat()
        }
    
    def stream_story(self, prompt: str) -> Generator[Dict[str, Any], None, None]:
        """Stream story generation."""
        
        result = self.story_module(prompt=prompt)
        
        yield {
            'type': 'story_start',
            'content': f'Creating a story about: {prompt}',
            'timestamp': datetime.now().isoformat()
        }
        
        for chunk in self._chunk_text(result.story):
            yield {
                'type': 'story',
                'content': chunk,
                'timestamp': datetime.now().isoformat()
            }
            time.sleep(self.delay)
        
        yield {
            'type': 'story_complete',
            'content': 'The End.',
            'timestamp': datetime.now().isoformat()
        }
    
    def stream_explanation(self, topic: str, audience: str = "general") -> Generator[Dict[str, Any], None, None]:
        """Stream technical explanation."""
        
        result = self.explanation_module(topic=topic, audience=audience)
        
        yield {
            'type': 'explanation_start',
            'content': f'Explaining {topic} for {audience} audience:',
            'timestamp': datetime.now().isoformat()
        }
        
        for chunk in self._chunk_text(result.explanation):
            yield {
                'type': 'explanation',
                'content': chunk,
                'timestamp': datetime.now().isoformat()
            }
            time.sleep(self.delay)
        
        yield {
            'type': 'explanation_complete',
            'content': '',
            'timestamp': datetime.now().isoformat()
        }
    
    def _chunk_text(self, text: str) -> Generator[str, None, None]:
        """Break text into streaming chunks."""
        words = text.split()
        for i in range(0, len(words), self.chunk_size):
            chunk = ' '.join(words[i:i + self.chunk_size])
            yield chunk
    
    def configure_streaming(self, chunk_size: int = 5, delay: float = 0.08):
        """Configure streaming parameters."""
        self.chunk_size = chunk_size
        self.delay = delay

# Initialize advanced streaming module
advanced_streaming = AdvancedStreamingModule()
print_result("Advanced streaming module initialized")

## Testing Advanced Streaming Features

In [ ]:
print_step("Testing Advanced Streaming", "Demonstrating structured streaming with metadata")

# Test 1: Streaming Q&A with reasoning
print("\n1. Streaming Q&A with Reasoning:")
print("-" * 40)

qa_question = "How do neural networks learn from data?"
print(f"Question: {qa_question}\n")

for stream_chunk in advanced_streaming.stream_qa(qa_question):
    if stream_chunk['type'] == 'reasoning_start':
        print(f"🤔 {stream_chunk['content']}")
    elif stream_chunk['type'] == 'reasoning':
        print(f"   {stream_chunk['content']}", end=" ")
    elif stream_chunk['type'] == 'answer_start':
        print(f"\n\n💡 {stream_chunk['content']}")
    elif stream_chunk['type'] == 'answer':
        print(f"{stream_chunk['content']}", end=" ", flush=True)
    elif stream_chunk['type'] == 'complete':
        print("\n\n✅ Response complete")

# Test 2: Streaming story generation
print("\n\n2. Streaming Story Generation:")
print("-" * 40)

story_prompt = "A robot learning to paint"
print(f"Story prompt: {story_prompt}\n")

for stream_chunk in advanced_streaming.stream_story(story_prompt):
    if stream_chunk['type'] == 'story_start':
        print(f"📖 {stream_chunk['content']}\n")
    elif stream_chunk['type'] == 'story':
        print(f"{stream_chunk['content']}", end=" ", flush=True)
    elif stream_chunk['type'] == 'story_complete':
        print(f"\n\n{stream_chunk['content']}")

# Test 3: Streaming explanation
print("\n\n3. Streaming Technical Explanation:")
print("-" * 40)

explanation_topic = "blockchain technology"
print(f"Topic: {explanation_topic}\n")

for stream_chunk in advanced_streaming.stream_explanation(explanation_topic, "beginner"):
    if stream_chunk['type'] == 'explanation_start':
        print(f"🎯 {stream_chunk['content']}\n")
    elif stream_chunk['type'] == 'explanation':
        print(f"{stream_chunk['content']}", end=" ", flush=True)
    elif stream_chunk['type'] == 'explanation_complete':
        print("\n\n✅ Explanation complete")

## Async Streaming Implementation

In [ ]:
class AsyncStreamingModule(dspy.Module):
    """Asynchronous streaming implementation for better performance."""
    
    def __init__(self):
        super().__init__()
        self.predictor = dspy.ChainOfThought("prompt -> response")
        self.chunk_size = 4
        self.delay = 0.05
    
    async def async_stream_response(self, prompt: str) -> AsyncGenerator[Dict[str, Any], None]:
        """Generate asynchronous streaming response."""
        
        # Start response generation
        yield {
            'type': 'start',
            'content': 'Processing your request...',
            'timestamp': datetime.now().isoformat()
        }
        
        # Simulate async processing
        await asyncio.sleep(0.1)
        
        try:
            # Generate response (in practice, this would be async)
            result = self.predictor(prompt=prompt)
            
            # Stream reasoning if available
            if hasattr(result, 'reasoning') and result.reasoning:
                yield {
                    'type': 'reasoning_start',
                    'content': 'Thinking...',
                    'timestamp': datetime.now().isoformat()
                }
                
                async for chunk in self._async_chunk_text(result.reasoning):
                    yield {
                        'type': 'reasoning',
                        'content': chunk,
                        'timestamp': datetime.now().isoformat()
                    }
            
            # Stream response
            yield {
                'type': 'response_start',
                'content': 'Response:',
                'timestamp': datetime.now().isoformat()
            }
            
            async for chunk in self._async_chunk_text(result.response):
                yield {
                    'type': 'response',
                    'content': chunk,
                    'timestamp': datetime.now().isoformat()
                }
            
            yield {
                'type': 'complete',
                'content': 'Done!',
                'timestamp': datetime.now().isoformat()
            }
            
        except Exception as e:
            yield {
                'type': 'error',
                'content': f'Error: {e}',
                'timestamp': datetime.now().isoformat()
            }
    
    async def _async_chunk_text(self, text: str) -> AsyncGenerator[str, None]:
        """Asynchronously chunk text for streaming."""
        words = text.split()
        for i in range(0, len(words), self.chunk_size):
            chunk = ' '.join(words[i:i + self.chunk_size])
            yield chunk
            await asyncio.sleep(self.delay)
    
    async def batch_stream_responses(self, prompts: List[str]) -> AsyncGenerator[Dict[str, Any], None]:
        """Stream responses for multiple prompts concurrently."""
        
        yield {
            'type': 'batch_start',
            'content': f'Processing {len(prompts)} prompts...',
            'timestamp': datetime.now().isoformat()
        }
        
        # Process prompts concurrently
        tasks = []
        for i, prompt in enumerate(prompts):
            task = asyncio.create_task(self._process_single_prompt(i, prompt))
            tasks.append(task)
        
        # Stream results as they complete
        for task in asyncio.as_completed(tasks):
            result = await task
            yield result
        
        yield {
            'type': 'batch_complete',
            'content': 'All prompts processed',
            'timestamp': datetime.now().isoformat()
        }
    
    async def _process_single_prompt(self, index: int, prompt: str) -> Dict[str, Any]:
        """Process a single prompt asynchronously."""
        
        start_time = time.time()
        
        try:
            result = self.predictor(prompt=prompt)
            
            return {
                'type': 'batch_result',
                'index': index,
                'prompt': prompt[:50] + '...' if len(prompt) > 50 else prompt,
                'response': result.response[:100] + '...' if len(result.response) > 100 else result.response,
                'duration': time.time() - start_time,
                'timestamp': datetime.now().isoformat()
            }
            
        except Exception as e:
            return {
                'type': 'batch_error',
                'index': index,
                'prompt': prompt[:50] + '...' if len(prompt) > 50 else prompt,
                'error': str(e),
                'timestamp': datetime.now().isoformat()
            }

# Initialize async streaming module
async_streaming = AsyncStreamingModule()
print_result("Async streaming module initialized")

## Testing Async Streaming

In [ ]:
print_step("Testing Async Streaming", "Demonstrating asynchronous response generation")

async def test_async_streaming():
    """Test asynchronous streaming functionality."""
    
    # Test 1: Single async stream
    print("\n1. Single Async Stream:")
    print("-" * 30)
    
    async_prompt = "Explain the benefits of asynchronous programming"
    print(f"Prompt: {async_prompt}\n")
    
    async for chunk in async_streaming.async_stream_response(async_prompt):
        if chunk['type'] == 'start':
            print(f"🚀 {chunk['content']}")
        elif chunk['type'] == 'reasoning_start':
            print(f"\n🤔 {chunk['content']}")
        elif chunk['type'] == 'reasoning':
            print(f"   {chunk['content']}", end=" ")
        elif chunk['type'] == 'response_start':
            print(f"\n\n💬 {chunk['content']}")
        elif chunk['type'] == 'response':
            print(f"{chunk['content']}", end=" ", flush=True)
        elif chunk['type'] == 'complete':
            print(f"\n\n✅ {chunk['content']}")
        elif chunk['type'] == 'error':
            print(f"\n❌ {chunk['content']}")
    
    # Test 2: Batch async processing
    print("\n\n2. Batch Async Processing:")
    print("-" * 30)
    
    batch_prompts = [
        "What is machine learning?",
        "Explain quantum computing",
        "Describe blockchain technology",
        "What is artificial intelligence?"
    ]
    
    print(f"Processing {len(batch_prompts)} prompts concurrently...\n")
    
    async for result in async_streaming.batch_stream_responses(batch_prompts):
        if result['type'] == 'batch_start':
            print(f"🔄 {result['content']}")
        elif result['type'] == 'batch_result':
            print(f"\n✅ Result {result['index'] + 1}:")
            print(f"   Prompt: {result['prompt']}")
            print(f"   Response: {result['response']}")
            print(f"   Duration: {result['duration']:.2f}s")
        elif result['type'] == 'batch_error':
            print(f"\n❌ Error {result['index'] + 1}:")
            print(f"   Prompt: {result['prompt']}")
            print(f"   Error: {result['error']}")
        elif result['type'] == 'batch_complete':
            print(f"\n🎉 {result['content']}")

# Run async tests
try:
    # Check if we're in a Jupyter environment
    import IPython
    # Use await in Jupyter
    await test_async_streaming()
except (ImportError, SyntaxError):
    # Run with asyncio in regular Python
    asyncio.run(test_async_streaming())
except Exception as e:
    print_error(f"Error in async streaming test: {e}")
    print("Note: Async features may require specific environment setup")

## Streaming Chat Interface

In [ ]:
class StreamingChatInterface:
    """Interactive streaming chat interface using DSPy."""
    
    def __init__(self):
        self.conversation_history = []
        self.chat_module = dspy.ChainOfThought("conversation_history, user_message -> assistant_response")
        self.chunk_size = 3
        self.delay = 0.06
    
    def add_message(self, role: str, content: str):
        """Add message to conversation history."""
        self.conversation_history.append({
            'role': role,
            'content': content,
            'timestamp': datetime.now().isoformat()
        })
    
    def get_context_string(self) -> str:
        """Get formatted conversation context."""
        if not self.conversation_history:
            return "This is the start of the conversation."
        
        context_parts = []
        for msg in self.conversation_history[-6:]:  # Last 6 messages
            context_parts.append(f"{msg['role']}: {msg['content']}")
        
        return "\n".join(context_parts)
    
    def stream_chat_response(self, user_message: str) -> Generator[Dict[str, Any], None, None]:
        """Generate streaming chat response."""
        
        # Add user message to history
        self.add_message('user', user_message)
        
        yield {
            'type': 'thinking',
            'content': 'Thinking...',
            'timestamp': datetime.now().isoformat()
        }
        
        try:
            # Generate response using conversation context
            context = self.get_context_string()
            result = self.chat_module(
                conversation_history=context,
                user_message=user_message
            )
            
            # Stream the response
            response_text = result.assistant_response
            
            yield {
                'type': 'response_start',
                'content': '',
                'timestamp': datetime.now().isoformat()
            }
            
            # Stream response in chunks
            words = response_text.split()
            for i in range(0, len(words), self.chunk_size):
                chunk = ' '.join(words[i:i + self.chunk_size])
                yield {
                    'type': 'response_chunk',
                    'content': chunk,
                    'timestamp': datetime.now().isoformat()
                }
                time.sleep(self.delay)
            
            # Add assistant response to history
            self.add_message('assistant', response_text)
            
            yield {
                'type': 'response_complete',
                'content': response_text,
                'timestamp': datetime.now().isoformat()
            }
            
        except Exception as e:
            error_msg = f"Sorry, I encountered an error: {e}"
            self.add_message('assistant', error_msg)
            
            yield {
                'type': 'error',
                'content': error_msg,
                'timestamp': datetime.now().isoformat()
            }
    
    def get_conversation_summary(self) -> Dict[str, Any]:
        """Get summary of the conversation."""
        
        user_messages = [msg for msg in self.conversation_history if msg['role'] == 'user']
        assistant_messages = [msg for msg in self.conversation_history if msg['role'] == 'assistant']
        
        return {
            'total_messages': len(self.conversation_history),
            'user_messages': len(user_messages),
            'assistant_messages': len(assistant_messages),
            'conversation_start': self.conversation_history[0]['timestamp'] if self.conversation_history else None,
            'last_activity': self.conversation_history[-1]['timestamp'] if self.conversation_history else None
        }
    
    def clear_history(self):
        """Clear conversation history."""
        self.conversation_history = []

# Initialize streaming chat interface
chat_interface = StreamingChatInterface()
print_result("Streaming chat interface initialized")

## Testing Streaming Chat

In [ ]:
print_step("Testing Streaming Chat", "Demonstrating conversational streaming interface")

# Simulate a conversation
chat_messages = [
    "Hello! Can you explain what DSPy is?",
    "That's interesting! How does it differ from LangChain?",
    "Can you give me a practical example of using DSPy?",
    "Thank you! This has been very helpful."
]

print("🤖 Starting streaming chat demonstration...\n")

for i, message in enumerate(chat_messages, 1):
    print(f"{'='*50}")
    print(f"Turn {i}")
    print(f"{'='*50}")
    
    print(f"👤 User: {message}")
    print("🤖 Assistant: ", end="")
    
    full_response = ""
    
    for chunk in chat_interface.stream_chat_response(message):
        if chunk['type'] == 'thinking':
            print("💭 ", end="", flush=True)
            time.sleep(0.5)
            print("\r🤖 Assistant: ", end="")
        elif chunk['type'] == 'response_chunk':
            print(chunk['content'], end=" ", flush=True)
            full_response += chunk['content'] + " "
        elif chunk['type'] == 'response_complete':
            print("\n")
        elif chunk['type'] == 'error':
            print(f"❌ {chunk['content']}\n")
    
    time.sleep(0.5)  # Pause between messages

# Show conversation summary
print("\n" + "="*60)
print("CONVERSATION SUMMARY")
print("="*60)

summary = chat_interface.get_conversation_summary()
print_result(f"Total messages: {summary['total_messages']}")
print_result(f"User messages: {summary['user_messages']}")
print_result(f"Assistant messages: {summary['assistant_messages']}")
print_result(f"Conversation duration: {summary['conversation_start']} to {summary['last_activity']}")

print("\n🎉 Streaming chat demonstration completed!")

## Performance Optimization for Streaming

In [ ]:
class OptimizedStreamingModule:
    """Performance-optimized streaming implementation."""
    
    def __init__(self):
        self.predictor = dspy.Predict("prompt -> response")
        
        # Performance tracking
        self.response_times = []
        self.chunk_counts = []
        self.total_requests = 0
        
        # Optimization settings
        self.adaptive_chunking = True
        self.min_chunk_size = 2
        self.max_chunk_size = 8
        self.base_delay = 0.05
    
    def optimized_stream(self, prompt: str) -> Generator[Dict[str, Any], None, None]:
        """Generate optimized streaming response."""
        
        start_time = time.time()
        self.total_requests += 1
        
        try:
            # Generate response
            result = self.predictor(prompt=prompt)
            response_text = result.response
            
            # Adaptive chunking based on content
            chunks = self._adaptive_chunk(response_text)
            
            # Stream with adaptive delay
            for i, chunk in enumerate(chunks):
                # Adaptive delay based on chunk position and size
                delay = self._calculate_adaptive_delay(i, len(chunks), len(chunk.split()))
                
                yield {
                    'type': 'chunk',
                    'content': chunk,
                    'chunk_index': i,
                    'total_chunks': len(chunks),
                    'delay': delay,
                    'timestamp': datetime.now().isoformat()
                }
                
                time.sleep(delay)
            
            # Track performance
            response_time = time.time() - start_time
            self.response_times.append(response_time)
            self.chunk_counts.append(len(chunks))
            
            yield {
                'type': 'complete',
                'content': 'Stream complete',
                'performance': {
                    'response_time': response_time,
                    'chunk_count': len(chunks),
                    'avg_chunk_size': len(response_text.split()) / len(chunks)
                },
                'timestamp': datetime.now().isoformat()
            }
            
        except Exception as e:
            yield {
                'type': 'error',
                'content': f'Streaming error: {e}',
                'timestamp': datetime.now().isoformat()
            }
    
    def _adaptive_chunk(self, text: str) -> List[str]:
        """Create adaptive chunks based on content structure."""
        
        if not self.adaptive_chunking:
            # Simple fixed-size chunking
            words = text.split()
            chunk_size = (self.min_chunk_size + self.max_chunk_size) // 2
            return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
        
        # Adaptive chunking based on sentence boundaries and content
        sentences = text.split('. ')
        chunks = []
        current_chunk = ""
        
        for sentence in sentences:
            sentence = sentence.strip() + '. ' if not sentence.endswith('.') else sentence + ' '
            
            # Check if adding this sentence would exceed max chunk size
            potential_chunk = current_chunk + sentence
            word_count = len(potential_chunk.split())
            
            if word_count <= self.max_chunk_size or not current_chunk:
                current_chunk = potential_chunk
            else:
                # Finalize current chunk
                if current_chunk.strip():
                    chunks.append(current_chunk.strip())
                current_chunk = sentence
        
        # Add remaining text
        if current_chunk.strip():
            chunks.append(current_chunk.strip())
        
        return chunks
    
    def _calculate_adaptive_delay(self, chunk_index: int, total_chunks: int, chunk_word_count: int) -> float:
        """Calculate adaptive delay based on chunk characteristics."""
        
        # Base delay
        delay = self.base_delay
        
        # Adjust for chunk size (larger chunks get slightly longer delay)
        delay += (chunk_word_count - self.min_chunk_size) * 0.01
        
        # Faster streaming at the beginning
        if chunk_index < 3:
            delay *= 0.8
        
        # Slightly slower at the end for dramatic effect
        if chunk_index >= total_chunks - 2:
            delay *= 1.2
        
        return max(0.02, min(0.15, delay))  # Clamp between 20ms and 150ms
    
    def get_performance_stats(self) -> Dict[str, Any]:
        """Get streaming performance statistics."""
        
        if not self.response_times:
            return {'status': 'no_data'}
        
        return {
            'total_requests': self.total_requests,
            'avg_response_time': sum(self.response_times) / len(self.response_times),
            'min_response_time': min(self.response_times),
            'max_response_time': max(self.response_times),
            'avg_chunk_count': sum(self.chunk_counts) / len(self.chunk_counts),
            'total_chunks_processed': sum(self.chunk_counts)
        }
    
    def configure_optimization(self, adaptive_chunking: bool = True, min_chunk_size: int = 2, 
                             max_chunk_size: int = 8, base_delay: float = 0.05):
        """Configure optimization parameters."""
        self.adaptive_chunking = adaptive_chunking
        self.min_chunk_size = min_chunk_size
        self.max_chunk_size = max_chunk_size
        self.base_delay = base_delay

# Initialize optimized streaming module
optimized_streaming = OptimizedStreamingModule()
print_result("Performance-optimized streaming module initialized")

## Testing Performance Optimization

In [ ]:
print_step("Testing Performance Optimization", "Comparing optimized vs standard streaming")

# Test different optimization settings
test_prompts = [
    "Explain machine learning algorithms in detail",
    "Describe the history of computer science",
    "What are the applications of artificial intelligence?",
]

print("\n1. Testing with Adaptive Chunking:")
print("-" * 40)

optimized_streaming.configure_optimization(adaptive_chunking=True)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\nTest {i}: {prompt}")
    print("Response: ", end="")
    
    chunk_details = []
    
    for chunk_data in optimized_streaming.optimized_stream(prompt):
        if chunk_data['type'] == 'chunk':
            print(chunk_data['content'], end=" ", flush=True)
            chunk_details.append({
                'size': len(chunk_data['content'].split()),
                'delay': chunk_data['delay']
            })
        elif chunk_data['type'] == 'complete':
            performance = chunk_data['performance']
            print(f"\n\n📊 Performance: {performance['response_time']:.2f}s, {performance['chunk_count']} chunks, avg {performance['avg_chunk_size']:.1f} words/chunk")
        elif chunk_data['type'] == 'error':
            print(f"\n❌ {chunk_data['content']}")

# Compare with fixed chunking
print("\n\n2. Testing with Fixed Chunking:")
print("-" * 40)

optimized_streaming.configure_optimization(adaptive_chunking=False)

test_prompt = "Explain the future of artificial intelligence and its potential impact on society"
print(f"Prompt: {test_prompt}")
print("Response: ", end="")

for chunk_data in optimized_streaming.optimized_stream(test_prompt):
    if chunk_data['type'] == 'chunk':
        print(chunk_data['content'], end=" ", flush=True)
    elif chunk_data['type'] == 'complete':
        performance = chunk_data['performance']
        print(f"\n\n📊 Performance: {performance['response_time']:.2f}s, {performance['chunk_count']} chunks")

# Show overall performance stats
print("\n\n3. Overall Performance Statistics:")
print("-" * 40)

stats = optimized_streaming.get_performance_stats()
if 'total_requests' in stats:
    print_result(f"Total Requests: {stats['total_requests']}")
    print_result(f"Average Response Time: {stats['avg_response_time']:.3f}s")
    print_result(f"Response Time Range: {stats['min_response_time']:.3f}s - {stats['max_response_time']:.3f}s")
    print_result(f"Average Chunk Count: {stats['avg_chunk_count']:.1f}")
    print_result(f"Total Chunks Processed: {stats['total_chunks_processed']}")
else:
    print_result("No performance data available")

print("\n🎯 Performance optimization demonstration completed!")

## Conclusion

This notebook demonstrated comprehensive streaming capabilities with DSPy:

### Key Features Implemented:

1. **Basic Streaming**: Simple word-by-word text generation
2. **Advanced Structured Streaming**: Metadata-rich streaming with different content types
3. **Async Streaming**: Non-blocking, concurrent response generation
4. **Interactive Chat**: Conversational streaming with memory
5. **Performance Optimization**: Adaptive chunking and delay management

### DSPy Integration:

- **Signatures**: Specialized streaming-optimized task definitions
- **Modules**: Composable streaming components
- **Chain of Thought**: Streaming reasoning and response generation
- **Memory Management**: Persistent conversation state

### Optimization Techniques:

- **Adaptive Chunking**: Content-aware chunk sizing
- **Dynamic Delays**: Context-sensitive streaming speed
- **Performance Monitoring**: Real-time metrics and optimization
- **Async Processing**: Concurrent request handling

### Real-world Applications:

- **Live Chat Systems**: Customer support and interactive assistants
- **Content Generation**: Real-time writing and creative applications
- **Educational Tools**: Interactive tutoring and explanation systems
- **Gaming**: Dynamic narrative and dialogue systems
- **Accessibility**: Real-time transcription and assistance

### Best Practices for Production:

1. **Error Handling**: Graceful degradation when streaming fails
2. **Performance Monitoring**: Track response times and user engagement
3. **Adaptive Configuration**: Adjust streaming parameters based on usage patterns
4. **Resource Management**: Efficient memory and network usage
5. **User Experience**: Balance speed with readability and comprehension

This streaming implementation showcases how DSPy enables building responsive, interactive AI applications that provide engaging real-time user experiences while maintaining the power and flexibility of the DSPy framework.